In [1]:
pip install cdsapi netCDF4 pandas geopandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
import cdsapi

c = cdsapi.Client(url = "https://cds.climate.copernicus.eu/api/v2", key = "255794:bedde892-0558-4ee4-b915-54e0fb7d698c")

c.retrieve(
    'sis-temperature-statistics',
    {
        'format': 'zip',
        'variable': 'average_temperature',
        'period': 'year',
        'statistic': 'time_average',
        'experiment': 'rcp4_5',
        'ensemble_statistic': 'ensemble_members_average',
    },
    'data/download.zip')

2023-10-03 11:00:11,933 INFO Welcome to the CDS
2023-10-03 11:00:11,936 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/sis-temperature-statistics
2023-10-03 11:00:12,097 INFO Request is completed
2023-10-03 11:00:12,105 INFO Downloading https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data3/dataset-sis-temperature-statistics-9bbcd136-11de-44f1-a449-cb4217885d92.zip to data/download.zip (40M)
2023-10-03 11:00:39,995 INFO Download rate 1.4M/s   


Result(content_length=41909241,content_type=application/zip,location=https://download-0007-clone.copernicus-climate.eu/cache-compute-0007/cache/data3/dataset-sis-temperature-statistics-9bbcd136-11de-44f1-a449-cb4217885d92.zip)

In [3]:
import zipfile

def unzip_file(zip_file_path, extract_to_folder):
    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to_folder)
        print(f"File '{zip_file_path}' has been successfully extracted to '{extract_to_folder}'.")
    except Exception as e:
        print(f"Error: {e}")

# Example usage:
zip_file_path = 'data/download.zip'  # Replace with the actual path to your ZIP file
extract_to_folder = 'data/'         # Replace with the folder where you want to extract the contents

unzip_file(zip_file_path, extract_to_folder)


File 'data/download.zip' has been successfully extracted to 'data/'.


In [4]:
import netCDF4 as nc
file = 'data/mean_Tmean_Yearly_rcp45_mean_v1.0.nc'
ds = nc.Dataset(file)

print(ds.variables.keys())

dict_keys(['mean_Tmean_Yearly', 'height', 'lat', 'lon', 'time'])


In [5]:
variable_name = 'mean_Tmean_Yearly'  # Replace with the name of the variable you want to access
data = ds.variables[variable_name][:]  # Access the data as a NumPy array


In [6]:
file = 'data/mean_Tmean_Yearly_rcp45_mean_v1.0.nc'
ds = nc.Dataset(file)

# List of variable names
variable_names = ds.variables.keys()

# Access and print the values of each variable
for variable_name in variable_names:
    variable_data = ds.variables[variable_name][:]  # Get the variable data
    print(f"Variable Name: {variable_name}")
   # print(f"Variable Data:\n{variable_data}\n")

Variable Name: mean_Tmean_Yearly
Variable Name: height
Variable Name: lat
Variable Name: lon
Variable Name: time


In [7]:
import numpy as np
import pandas as pd

In [ ]:
# Define the number of samples you want
num_samples = 9000000  # Adjust this number as needed

# Get the dimensions of latitude, longitude, and time
num_latitudes = len(ds.dimensions['lat'])
num_longitudes = len(ds.dimensions['lon'])
num_times = len(ds.dimensions['time'])

print(num_latitudes)
print(num_longitudes)
print(num_times)
# Create random indices for latitude, longitude, and time
random_lat_indices = np.random.choice(num_latitudes, num_samples)
random_lon_indices = np.random.choice(num_longitudes, num_samples)
random_time_indices = np.random.choice(num_times, num_samples)

# Create empty lists to store data
lat_list, lon_list, time_list, mean_Tmean_list = [], [], [], []

# Iterate through the selected random indices and store the corresponding data
for lat_idx, lon_idx, time_idx in zip(random_lat_indices, random_lon_indices, random_time_indices):
    lat_val = ds.variables['lat'][lat_idx]
    lon_val = ds.variables['lon'][lon_idx]
    time_val = ds.variables['time'][time_idx]
    mean_Tmean_val = ds.variables['mean_Tmean_Yearly'][time_idx, lat_idx, lon_idx]

    lat_list.append(lat_val)
    lon_list.append(lon_val)
    time_list.append(time_val)
    mean_Tmean_list.append(mean_Tmean_val)

# Create a Pandas DataFrame
data = {
    'lat': lat_list,
    'lon': lon_list,
    'time': time_list,
    'mean_Tmean_Yearly': mean_Tmean_list
}
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

# Close the NetCDF file
ds.close()

425
599
100


In [ ]:
output_csv_file = 'output.csv'
df.to_csv(output_csv_file, index=False)

In [ ]:
df.head()

In [ ]:
datatypes = df.dtypes
datatypes

In [ ]:
df['lat']=df['lat'].astype(float)
df['lon']=df['lon'].astype(float)
df['time']=df['time'].astype(float)
df['mean_Tmean_Yearly']=df['mean_Tmean_Yearly'].astype(float)
datatypes = df.dtypes
datatypes

In [ ]:
missing_values = df.isna().sum()
print(missing_values)

In [ ]:
df = df.dropna()
missing_values = df.isna().sum()
print(missing_values)

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="mean_Tmean_Yearly")
fig.show()

In [ ]:
import plotly.express as px
fig = px.histogram(df, x="lon")
fig.show()